In [6]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import confusion_matrix, classification_report

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'omar-vargas-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'Iris.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
iris = pd.read_csv(file_content_stream)
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [7]:
## Changing labels to numbers
iris['Species_numb'] = np.where(iris['Species'] == 'Iris-virginica', 1,
                               np.where(iris['Species'] == 'Iris-versicolor', 2, 3))

In [8]:
## Defining inputs and target
X = iris.drop(columns = ['Id', 'Species', 'Species_numb'], axis = 1)
Y = iris['Species_numb']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [9]:
#Min-Max transformation
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

One-vs-One (Log Reg)

In [11]:
one_vs_one_LR = OneVsOneClassifier(estimator = LogisticRegression()).fit(X_train, Y_train)

## Predicting on test
one_vs_one_LR_pred = one_vs_one_LR.predict(X_test)

print(confusion_matrix(Y_test, one_vs_one_LR_pred))
print(classification_report(Y_test, one_vs_one_LR_pred))

[[ 8  2  0]
 [ 0 10  0]
 [ 0  0 10]]
              precision    recall  f1-score   support

           1       1.00      0.80      0.89        10
           2       0.83      1.00      0.91        10
           3       1.00      1.00      1.00        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30



One-vs-One (Decision Tree)

In [12]:
one_vs_one_tree = OneVsOneClassifier(estimator = DecisionTreeClassifier(max_depth = 3)).fit(X_train, Y_train)

## Predicting on test
one_vs_one_tree_pred = one_vs_one_tree.predict(X_test)

print(confusion_matrix(Y_test, one_vs_one_tree_pred))
print(classification_report(Y_test, one_vs_one_tree_pred))

[[ 9  1  0]
 [ 2  8  0]
 [ 0  0 10]]
              precision    recall  f1-score   support

           1       0.82      0.90      0.86        10
           2       0.89      0.80      0.84        10
           3       1.00      1.00      1.00        10

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30



In [ ]:
## Based on my results, I would use the Logistic Regression model to predict iris species as the average 
## precision and recall are higher.